In [ ]:
pip install mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp
import numpy as np  
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


mp_drawing = mp.solutions.drawing_utils  # get drawing utilities from MediaPipe
mp_pose = mp.solutions.pose  # grabbing the Pose model from MediaPipe

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First point
    b = np.array(b) # Mid point
    c = np.array(c) # End point
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [4]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
  
    return pose

In [6]:
actions = np.array(['bicep_curls', 'chest_press', 'y_row', 'w_row'])

In [7]:
label_map = {label:num for num, label in enumerate(actions)}

In [8]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join('MP_Data', action))).astype(int):
        window = []
        for frame_num in range(30):
            res = np.load(os.path.join('MP_Data', action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [9]:
model = Sequential()
model.add(LSTM(64,return_sequences=True,activation = 'relu', input_shape= (30,132)))
model.add(LSTM(128,return_sequences=True,activation = 'relu'))
model.add(LSTM(64,return_sequences=False,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(actions.shape[0],activation = 'softmax'))

In [10]:
model.compile(optimizer='Adam',loss = 'categorical_crossentropy', metrics=['categorical_accuracy'])

In [11]:
model.load_weights('trx.h5')

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            50432     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 4)                 132       
                                                                 
Total params: 205,028
Trainable params: 205,028
Non-tr

In [29]:
cap = cv2.VideoCapture(0)


# For the model
sequence = []
threshold = 0.3

# Booleans for Hot-Keys
running = False
angle_view = False
wire_view = True
message_view = False

#Boolean for Auto-Detect
auto = False

# Manage the active exercise
# 0 - Chest Press
# 1 - Y Row
# 2 - W Row
# 3 - Biceps

active = 0

## All of the calculations run in this loop.
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.6) as pose:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        k = cv2.waitKey(33) # THIS GRABS THE KEY VALUE
        
        ## Hot keys for switching heurisitics.

        # Show the wireframe
        if(k == 116):
            running = True
            message_view = True
            wire_view = True
        
        # Turn off the wireframe.
        if(k == 121):
            running = False

        if(k == 103):
            angle_view = True if not angle_view else False
        
        if(k == 104):
            message_view = True if not message_view else False

        if(k == 119):
            wire_view = True if not wire_view else False

        if(k == 122):
            active = 0

        if(k == 120):
            active = 1

        if(k == 99):
            active = 2

        if(k == 118):
            active = 3

        # Press the A key to activate autoprediction
        if(k == 97):
            auto = True if not auto else False
            active = -1
        
        if(running):
            try:

                # draw the rectangle for the exercise title
                cv2.rectangle(image,(850,0),(1290,100),(220,220,220),-1)

                landmarks = results.pose_landmarks.landmark
                
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                ls_visibility = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                rs_visibility = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
                
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                le_visibility = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                re_visibility = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
                
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                rw_visibility = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                lw_visibility = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
                
                ## This is the automatic mode for using the model to switch heuristics
                if auto:
                    try:
                        keypoints = extract_keypoints(results)
                    except:
                        print("Something went wrong.")
                    sequence.append(keypoints)
                    sequence = sequence[-30:]

                    ## If we have 30 frames, make a predicition.
                    if len(sequence) == 30:
                        res = model.predict(np.expand_dims(sequence, axis=0))[0]
                       

                    if res[np.argmax(res)] > threshold:
                        pred_pose = np.argmax(res)

                        if pred_pose == 0:
                            active = 3
                           
                        
                        elif pred_pose == 1:
                            active = 0
                           

                        elif pred_pose == 2:
                            active = 2
                          

                        elif pred_pose == 3:
                            active = 1
                         


                    else: 
                        pred_pose = -1
                          
                ## Calculate Angles
                
                left_knee_angle = calculate_angle(left_ankle, left_knee, left_hip)
                right_knee_angle = calculate_angle(right_ankle, right_knee, right_hip)
                
                right_hip_angle = calculate_angle(right_knee, right_hip, right_shoulder)
                left_hip_angle = calculate_angle(left_knee, left_hip, left_shoulder)
                
                left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
                right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
                
                l_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                r_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                
                l_alt_angle = calculate_angle(right_shoulder, left_shoulder, left_elbow)
                r_alt_angle = calculate_angle(left_shoulder, right_shoulder, right_elbow)
                
                left_arm_visible = ls_visibility>0.8 and le_visibility>0.8 and lw_visibility>0.8
                right_arm_visible = rs_visibility>0.8 and re_visibility>0.8 and rw_visibility>0.8
                
                if message_view:
                    cv2.rectangle(image,(1,620),(1400,720),(220,220,220),-1)

                ##############################
                ## Y - Row
                if(active == 1):

                    cv2.putText(image,"Y Row", (955,65), cv2.FONT_HERSHEY_TRIPLEX, 2, (10, 0, 0), 2)
                    cv2.putText(image,"Visibility", (850,665), cv2.FONT_HERSHEY_TRIPLEX, 1, (10, 0, 0), 2)
                    cv2.putText(image,"Angle", (20,665), cv2.FONT_HERSHEY_TRIPLEX, 1, (10, 0, 0), 2)
                    cv2.putText(image,"Straight Arms", (420,665), cv2.FONT_HERSHEY_TRIPLEX, 1, (10, 0, 0), 2)
                    cv2.putText(image,"Left Arm", (420,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (10, 0, 0), 1)
                    cv2.putText(image,"Right Arm", (570,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (10, 0, 0), 1)
                    cv2.putText(image,"Right Side", (155,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (10,0,0), 1)
                    cv2.putText(image,"Left Side", (20,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (10,0,0), 1)


                    ## To see if the core is straight and and if legs are straight
                    if left_knee_angle > 170 and right_knee_angle > 170 and left_hip_angle > 150 and right_hip_angle > 150:
                        isCorrect = "True"
                        color = (0,255,0)
                    else:
                        isCorrect = "False"
                        color = (0,0,255)
                    


                    if (left_arm_visible):
                        cv2.putText(image,"Left Arm", (850,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (0, 100, 0), 1)
                        #checking if left arm is straight
                        if l_angle>90:
                            isCorrect = "True"
                            color = (0,140,0)
                        else:
                            isCorrect = "False"
                            color = (0,0,200)
                        cv2.putText(image,"Left Side", (20,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                    
                        # Checking if the angle between shoulder and left arm is correct
                        # Angle between left arm angle and left shoulder
                        if l_alt_angle>180 and l_alt_angle<160:
                            isCorrect = "True"
                            color = (0,120,0)
                        else:
                            isCorrect = "False"
                            color = (0,0,255)
                    
                        cv2.putText(image,"Left Arm", (420,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                    
                    else:
                        cv2.putText(image,"Left Arm", (850,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (250, 0, 0), 1)
                
                
                    if (right_arm_visible):
                        
                        cv2.putText(image,"Right Arm", (1000,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (0, 100, 0), 1)

                        #checking if right arm is straight
                        if r_angle>90:
                            isCorrect = "True"
                            color = (0,140,0)
                        else:
                            isCorrect = "False"
                            color = (0,0,200)
                        cv2.putText(image,"Right Side", (155,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                        
                        #Checking if the angle between shoulder and left arm is correct
                        # angle between right shoulder and right arm
                        if r_alt_angle>160 and r_alt_angle<180:
                            isCorrect = "True"
                            color = (0,100,0)
                        else:
                            isCorrect = "False"
                            color = (0,0,255)
                        cv2.putText(image,"Right Arm", (570,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                        
                    else:
                        cv2.putText(image,"Right Arm", (1000,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (250, 0, 0), 1)
                

                    if(angle_view):
                        cv2.rectangle(image,(0,0),(400,500),(220,220,220),-1)
                        cv2.putText(image, "Left Side", (10,60), cv2.FONT_HERSHEY_SIMPLEX, 1.45, (0,0,0), 3)
                        cv2.putText(image, "L Elbow | L Shoulder | R Shoulder", (10,105), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(l_alt_angle,2)), (10,135), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)
                        cv2.putText(image, "L Wrist | L Elbow | L Shoulder", (10,170), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(l_angle,2)), (10,200), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)

                        cv2.putText(image, "Right Side", (10,290), cv2.FONT_HERSHEY_SIMPLEX, 1.45, (0,0,0), 3)
                        cv2.putText(image, "R Elbow | R Shoulder | L Shoulder", (10,335), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(r_alt_angle,2)), (10,365), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)
                        cv2.putText(image, "R Wrist | R Elbow | R Shoulder", (10,405), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(r_angle,2)), (10,440), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)
                        
               
                
                ###########################
                ## CHEST PRESS
    
                if(active == 0):
                
                    cv2.putText(image,"Chest Press", (865,65), cv2.FONT_HERSHEY_TRIPLEX, 2, (10, 0, 0), 2)
                    cv2.putText(image,"Visibility", (850,665), cv2.FONT_HERSHEY_TRIPLEX, 1, (10, 0, 0), 2)
                    cv2.putText(image,"Angle", (20,665), cv2.FONT_HERSHEY_TRIPLEX, 1, (10, 0, 0), 2)
                    cv2.putText(image,"Parallel Shoulders", (420,665), cv2.FONT_HERSHEY_TRIPLEX, 1, (10, 0, 0), 2)
                    
                    
                    cv2.putText(image,"Right Side", (155,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (10,0,0), 1)
                    cv2.putText(image,"Left Side", (20,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (10,0,0), 1)
                    frame_h = frame.shape[0]

                    if left_knee_angle>170 and right_knee_angle>170 and left_hip_angle>150 and right_hip_angle>150:
                        isCorrect = "True"
                        color = (0,255,0)
                    else:
                        isCorrect = "False"
                        color = (0,0,255)
                    
                            
                #caculating the height difference between shoulders
                    shoulder_Y_diff =  abs(int(left_shoulder[1]*frame_h) - int(right_shoulder[1]*frame_h))
                
                
                #checking if shoulders are in parallel position
                    if shoulder_Y_diff>10:
                        color = (0,0,255)
                        if int(left_shoulder[1]*frame_h)>int(right_shoulder[1]*frame_h):
                            upper_shoulder = "Left shoulder"
                        else:
                            upper_shoulder = "Right Shoulder"
                        cv2.putText(image,"Shoulders are not parallel", (420,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (0, 0, 190), 1)
                    else:
                        color = (0,140,0)
                        cv2.putText(image,"Shoulders are parallel", (420,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                    
                   
                    if (left_arm_visible):
                        cv2.putText(image, "Left Arm", (850,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (0, 150, 0), 1)
                        #checking if left arm is in correct form
                        if l_angle < 180 and l_angle > 80:
                            isCorrect = "True"
                            color = (0,140,0)
                        else:
                            isCorrect = "False"
                            color = (0,0,200)
                        cv2.putText(image,"Left Side", (20,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                        
                    else:
                        cv2.putText(image, "Left Arm", (850,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (190, 0, 0), 1)
                        
        
                    if (right_arm_visible):
                        cv2.putText(image,"Right Arm", (1000,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (0, 140, 0), 1)

                        #checking if right arm is in correct form
                        if r_angle<180 and r_angle> 80:
                            isCorrect = "True"
                            color = (0,140,0)
                        else:
                            isCorrect = "False"
                            color = (0, 0,255)
                        cv2.putText(image,"Right Side", (155,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                        
                        
                    else:
                        cv2.putText(image,"Right Arm", (1000,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (190, 0, 0), 1) 

                    if(angle_view):
                        cv2.rectangle(image,(0,0),(400,500),(220,220,220),-1)
                        cv2.putText(image, "Left Side", (10,60), cv2.FONT_HERSHEY_SIMPLEX, 1.45, (0,0,0), 3)
                        cv2.putText(image, "L Elbow | L Shoulder | R Shoulder", (10,105), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(l_alt_angle,2)), (10,135), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)
                        cv2.putText(image, "L Wrist | L Elbow | L Shoulder", (10,170), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(l_angle,2)), (10,200), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)

                        cv2.putText(image, "Right Side", (10,290), cv2.FONT_HERSHEY_SIMPLEX, 1.45, (0,0,0), 3)
                        cv2.putText(image, "R Elbow | R Shoulder | L Shoulder", (10,335), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(r_alt_angle,2)), (10,365), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)
                        cv2.putText(image, "R Wrist | R Elbow | R Shoulder", (10,405), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(r_angle,2)), (10,440), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)
                    
                
                
                ##############################
                ## W-POSE
                if(active == 2):

                    cv2.putText(image,"W Row", (955,65), cv2.FONT_HERSHEY_TRIPLEX, 2, (10, 0, 0), 2)
                    cv2.putText(image,"Visibility", (850,665), cv2.FONT_HERSHEY_TRIPLEX, 1, (10, 0, 0), 2)
                    cv2.putText(image,"Angle", (20,665), cv2.FONT_HERSHEY_TRIPLEX, 1, (10, 0, 0), 2)
                    cv2.putText(image,"Arm Height", (420,665), cv2.FONT_HERSHEY_TRIPLEX, 1, (10, 0, 0), 2)
                    cv2.putText(image,"Left Arm", (420,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (10, 0, 0), 1)
                    cv2.putText(image,"Right Arm", (570,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (10, 0, 0), 1)
                    
                    cv2.putText(image,"Right Side", (155,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (10,0,0), 1)
                    cv2.putText(image,"Left Side", (20,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (10,0,0), 1)
                    
                    
                        
                    if (left_arm_visible):
                        cv2.putText(image,"Left Arm", (850,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (0, 100, 0), 1)
                        #checking if left arm is straight
                        if l_angle<180 and l_angle> 70:
                            isCorrect = "True"
                            color = (0,140,0)
                        else:
                            isCorrect = "False"
                            color = (0,0,190)
                        cv2.putText(image,"Left Side", (20,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                        
                        #Checking if the angle between shoulder and left arm is correct
                        if l_alt_angle>160 and l_alt_angle<180:
                            isCorrect = "True"
                            color = (0,140,0)
                        else:
                            isCorrect = "False"
                            color = (0,0,190)

                        cv2.putText(image,"Left Arm", (420,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                        
                    else:
                        cv2.putText(image, "Left Arm", (850,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (190, 0, 0), 1)
                    
                    
                    if (right_arm_visible):
                        cv2.putText(image,"Right Arm", (1000,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (0, 140, 0), 1) 
                        #checking if right arm is straigvht
                        if r_angle<180 and r_angle> 70:
                            isCorrect = "True"
                            color = (0,140,0)
                        else:
                            isCorrect = "False"
                            color = (0, 0,200)
                        cv2.putText(image,"Right Side", (155,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                        
                        #Checking if the angle between shoulder and left arm is correct
                        if r_alt_angle>160 and r_alt_angle<180:
                            isCorrect = "True"
                            color = (0,140,0)
                        else:
                            isCorrect = "False"
                            color = (0,0,190)
                        cv2.putText(image,"Right Arm", (570,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                        
                    else:
                        cv2.putText(image,"Right Arm", (1000,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (190, 0, 0), 1)  

                    if(angle_view):
                        cv2.rectangle(image,(0,0),(400,500),(220,220,220),-1)
                        cv2.putText(image, "Left Side", (10,60), cv2.FONT_HERSHEY_SIMPLEX, 1.45, (0,0,0), 3)
                        cv2.putText(image, "L Elbow | L Shoulder | R Shoulder", (10,105), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(l_alt_angle,2)), (10,135), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)
                        cv2.putText(image, "L Wrist | L Elbow | L Shoulder", (10,170), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(l_angle,2)), (10,200), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)

                        cv2.putText(image, "Right Side", (10,290), cv2.FONT_HERSHEY_SIMPLEX, 1.45, (0,0,0), 3)
                        cv2.putText(image, "R Elbow | R Shoulder | L Shoulder", (10,335), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(r_alt_angle,2)), (10,365), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)
                        cv2.putText(image, "R Wrist | R Elbow | R Shoulder", (10,405), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(r_angle,2)), (10,440), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)


                ##############################
                ## Biceps
                if(active == 3):
                    
                    cv2.putText(image,"Bicep Curls", (865,65), cv2.FONT_HERSHEY_TRIPLEX, 2, (10, 0, 0), 2)
                    cv2.putText(image,"Visibility", (850,665), cv2.FONT_HERSHEY_TRIPLEX, 1, (10, 0, 0), 2)
                    cv2.putText(image,"Angle", (20,665), cv2.FONT_HERSHEY_TRIPLEX, 1, (10, 0, 0), 2)
                    cv2.putText(image,"Tight Elbows", (420,665), cv2.FONT_HERSHEY_TRIPLEX, 1, (10, 0, 0), 2)
                    cv2.putText(image,"Left Arm", (420,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (10, 0, 0), 1)
                    cv2.putText(image,"Right Arm", (570,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (10, 0, 0), 1)
                    
                    cv2.putText(image,"Right Side", (155,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (10,0,0), 1)
                    cv2.putText(image,"Left Side", (20,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (10,0,0), 1)

                    l_alt_angle = calculate_angle(right_elbow, right_shoulder, right_hip)
                    r_alt_angle = calculate_angle(left_elbow, left_shoulder, left_hip)  
                    if (left_arm_visible):
                        cv2.putText(image,"Left Arm", (850,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (0, 100, 0), 1)
                        
                        #checking if left arm is straight
                        if l_angle < 180 and l_angle > 20:
                            isCorrect = "True"
                            color = (0,140,0)
                        else:
                            isCorrect = "False"
                            color = (0,0,190)
                        cv2.putText(image,"Left Side", (20,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                        
                        #Checking if the angle between shoulder and left arm is correct
                        if l_alt_angle > 0 and l_alt_angle < 35:
                            isCorrect = "True"
                            color = (0,140,0)
                        else:
                            isCorrect = "False"
                            color = (0,0,190)
                        cv2.putText(image,"Left Arm", (420,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                        
                    else:
                        cv2.putText(image,"Left Arm", (850,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (190, 0, 0), 1)
                    
                    
                    if (right_arm_visible):
                        cv2.putText(image,"Right Arm", (1000,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (0, 100, 0), 1)
                        #checking if right arm is straight
                        if r_angle<180 and r_angle> 20:
                            isCorrect = "True"
                            color = (0,140,0)
                        else:
                            isCorrect = "False"
                            color = (0, 0,190)
                        cv2.putText(image,"Right Side", (155,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                        
                        #Checking if the angle between shoulder and left arm is correct
                        if r_alt_angle>0 and r_alt_angle<35:
                            isCorrect = "True"
                            color = (0,140,0)
                        else:
                            isCorrect = "False"
                            color = (0,0,190)
                        cv2.putText(image,"Right Arm", (570,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, color, 1)
                        
                    else:
                        cv2.putText(image,"Right Arm", (1000,700), cv2.FONT_HERSHEY_TRIPLEX, 0.75, (190, 0, 0), 1) 

                    if(angle_view):
                        cv2.rectangle(image,(0,0),(400,500),(220,220,220),-1)
                        cv2.putText(image, "Left Side", (10,60), cv2.FONT_HERSHEY_SIMPLEX, 1.45, (0,0,0), 3)
                        cv2.putText(image, "L Elbow | L Shoulder | R Shoulder", (10,105), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(l_alt_angle,2)), (10,135), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)
                        cv2.putText(image, "L Wrist | L Elbow | L Shoulder", (10,170), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(l_angle,2)), (10,200), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)

                        cv2.putText(image, "Right Side", (10,290), cv2.FONT_HERSHEY_SIMPLEX, 1.45, (0,0,0), 3)
                        cv2.putText(image, "R Elbow | R Shoulder | L Shoulder", (10,335), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(r_alt_angle,2)), (10,365), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)
                        cv2.putText(image, "R Wrist | R Elbow | R Shoulder", (10,405), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 2)
                        cv2.putText(image, str(round(r_angle,2)), (10,440), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (150,0,0), 2)
  
            except:
                pass
            
            
            if(wire_view):
            # Render detections
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(145,117,66), thickness=2, circle_radius=2), 
                                        mp_drawing.DrawingSpec(color=(100,66,230), thickness=2, circle_radius=2) 
                                        )               
            
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    
    cap.release()
    
    cv2.destroyAllWindows()

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
0
0
0
0
0
0
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
1
1
1
1
1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
